In [2]:
from ast import literal_eval
dataFromFile = {}
addresses = []
with open("inputData/ourData3.txt", "r") as f:
    for line in f:
        key = literal_eval(line[:line.find(")")+1])
        addresses.append(key)
        value = literal_eval(line[line.find(")")+1:])
        dataFromFile[key] = value

In [134]:
limit = 0
for i in dataFromFile:
    print dataFromFile[i]
    if limit > 10: break
    limit += 1

['Duplex', '40.848794', '-73.920395', '2016', '1993', '2000', '2640', '11/01/2005', '385000', '527411', '11/14/2017', '13574', '564330', '564330', '0', '10453']
['SingleFamily', '40.834695', '-73.86712', '2016', '1928', '2439', '2400', '11/14/2007', '600000', '507220', '11/14/2017', '8732', '547798', '547798', '0', '10472']
['Duplex', '40.846745', '-73.82921', '2016', '1925', '3300', '2300', '03/22/2005', '545500', '539127', '11/14/2017', '-407', '566083', '566083', '0', '10461']
['Quadruplex', '40.871585', '-73.859794', '2016', '1920', '5000', '2149', '03/14/2010', '475000', '491510', '11/14/2017', '-14008', '525916', '525916', '0', '10469']
['Quadruplex', '40.831064', '-73.87563', '2016', '1927', '2500', '3321', '03/09/2012', '750000', '640223', '11/14/2017', '3056', '678636', '678636', '0', '10472']
['Triplex', '40.8915', '-73.84869', '2016', '1930', '2500', '2310', '01/04/2000', '270000', '567947', '11/14/2017', '-637', '607703', '607703', '0', '10466']
['MultiFamily2To4', '40.8470

In [118]:
#Partition X and Y
xData = []
yData = []
for key in dataFromFile:
    featuresAndLabel = list(key) + dataFromFile[key]
    xData.append(featuresAndLabel[:-7] + featuresAndLabel[-6:])
    yData.append(int(featuresAndLabel[-7]))

In [119]:
#Remove Address and State
newXData = []
for x in xData:
    newDp = x[2:]
    newXData.append(newDp)
xData = newXData

In [122]:
import numpy as np
from sklearn.model_selection import KFold
X = np.array(xData)
y = np.array(yData)
kf = KFold(n_splits=4)
kf.get_n_splits(X)
KFold(n_splits=2, random_state=None, shuffle=False)
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [123]:
def RMSD(predVals, trueVals):
    sum = 0
    for i in range(len(trueVals)):
        sum += (trueVals[i] - predVals[i])**2
    return sum**0.5

In [157]:
from sklearn import linear_model
linear_model = linear_model.LinearRegression()
linear_model.fit(X_train, y_train)
print RMSD(linear_model.predict(xData), yData)/(len(yData))

with open("ouputs.txt", "w") as f:
    outputs = linear_model.predict(xData)
    for i in range(len(addresses)):
        f.write(str(addresses[i]) + "\t" + str(outputs[i])+"\n")

4683.93471781


In [125]:
from sklearn import linear_model
lasso_model = linear_model.Lasso(alpha = 0.1)
lasso_model.fit(X_train, y_train)
print RMSD(lasso_model.predict(xData), yData)/(len(yData))

4683.93302221


In [126]:
from sklearn import linear_model
elastic_model = linear_model.ElasticNet()
elastic_model.fit(X_train, y_train)
print RMSD(elastic_model.predict(xData), yData)/(len(yData))

4678.39707971


In [130]:
from sklearn import linear_model
ridge_model = linear_model.BayesianRidge()
ridge_model.fit(X_train, y_train)
print RMSD(ridge_model.predict(xData), yData)/(len(yData))

4736.9427357
198


In [128]:
#print lasso_model.coef_

In [141]:
from sklearn import linear_model
lasso_model = linear_model.Lasso(alpha = 0.1)
lasso_model.fit(X_train, y_train)
def getPredictions(testPoints, regressionType, xData, yData):
    print lasso_model.predict([xData[0],xData[2],xData[3],xData[4],xData[5]])

In [149]:
testPoints = [
    ('1635 MONTGOMERY AVE', 'New York'),
    ('1721 MAHAN AVE', 'New York'),
    ('931 BURKE AVE', 'New York'),
    ('1256 STRATFORD AVE', 'New York'),
    ('1774 EDENWALD AVE', 'New York')
]
getPredictions(testPoints, 'lasso', X_train, y_train)

[ 543000.18840265  541543.66917074  514691.65774091  655837.99030106
  566694.64984344]
